In [ ]:
import ROOT
import fastjet as fj
import fjext
import fjcontrib
import fjtools

import pythia8
import pythiafjext
import pythiaext
from heppy.pythiautils import configuration as pyconf

from tqdm.notebook import tqdm
import argparse
import os
import sys

from __future__ import absolute_import, division, print_function

# standard numerical library imports
import numpy as np
rng = np.random.RandomState(0)

# matplotlib is required for this example
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (5,4.5)
from matplotlib import animation, rc

from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 200em; }</style>"))

In [ ]:
def get_args_from_settings(ssettings):
    sys.argv=[' '] + ssettings.split()
    parser = argparse.ArgumentParser(description='pythia8 fastjet on the fly')
    pyconf.add_standard_pythia_args(parser)
    parser.add_argument('--output', default="test_ang_ue.root", type=str)
    parser.add_argument('--user-seed', help='pythia seed', default=1111, type=int)
    args = parser.parse_args()
    return args

In [ ]:
mycfg = []
ssettings = "--py-ecm 5000 --user-seed=100000 --nev 1000"
args = get_args_from_settings(ssettings)
pythia_hard = pyconf.create_and_init_pythia_from_args(args, mycfg)

In [ ]:
max_eta_hadron=1
parts_selector_h = fj.SelectorAbsEtaMax(max_eta_hadron)
jet_R0 = 0.4
jet_selector = fj.SelectorPtMin(80.0) & fj.SelectorPtMax(120.0) & fj.SelectorAbsEtaMax(max_eta_hadron - 1.05 * jet_R0)
n_pileup = 1

In [ ]:
# print the banner first
fj.ClusterSequence.print_banner()
print()
# set up our jet definition and a jet selector
jet_R0 = 0.4
jet_def = fj.JetDefinition(fj.antikt_algorithm, jet_R0)
print(jet_def)

sj_def = fj.JetDefinition(fj.antikt_algorithm, 0.1)
print(sj_def)

In [ ]:
# for n in tqdm(range(args.nev)):
def next_event():
    while (1):
        if not pythia_hard.next():
            continue
        parts_pythia_h = pythiafjext.vectorize_select(pythia_hard, [pythiafjext.kFinal], 0, False)
        parts_pythia_h_selected = parts_selector_h(parts_pythia_h)

        mult_hard = len(parts_pythia_h_selected)

        jets_h = fj.sorted_by_pt(jet_selector(jet_def(parts_pythia_h_selected)))

        if len(jets_h) < 1:
            continue

        j = jets_h[0]

        #make the subjets
        subjets = fj.sorted_by_pt(sj_def(j.constituents()))
        if len(subjets) < 3:
            continue
        return j, subjets
    

In [ ]:
def draw_jet(j):
    # for the jet but not subjets
    pts = [p.perp() for p in fj.sorted_by_pt(j.constituents())]
    ys = [j.rapidity() - p.rapidity() for p in fj.sorted_by_pt(j.constituents())]
    phis = [j.delta_phi_to(p) for p in fj.sorted_by_pt(j.constituents())]
    
    phis.append(0.4)
    phis.append(-0.4)
    ys.append(0.4)
    ys.append(-0.4)
    pts.append(0)
    pts.append(0)
    zs = [pt/j.perp() for pt in pts]
    zs_sized = [z*1000. for z in zs]
    cs = [int(z*100.) for z in zs]

    plt.figure()
    # plt.scatter(phis, ys, c=colors, s=zs_sized, alpha=0.4, cmap="PuOr") #cmap='viridis')
    plt.scatter(phis, ys, c=cs, s=zs_sized, alpha=0.4, cmap="magma") #cmap='viridis')
    
    plt.xlabel('phi')
    plt.ylabel('y')
    plt.colorbar();  # show color scale
    plt.show()
    #plt.rcdefaults()

In [ ]:
def draw_subjets(j, sjs):
    # for subjets
    pts = []
    ys = []
    phis = []
    cs = []
    for i,sj in enumerate(sjs):
        sc = fj.sorted_by_pt(sj.constituents())
        pts.extend([p.perp() for p in sc])
        ys.extend([j.rapidity() - p.rapidity() for p in sc])
        phis.extend([j.delta_phi_to(p) for p in sc])
        if i == 0:
            for p in sc:
                cs.append([1,0,0,0.3])
        else:
            gr_scale = (len(sjs)-i)/len(sjs)
            gr_col = 0.4-0.3*gr_scale
            # print(gr_col)
            for p in sc:
                cs.append([gr_col, gr_col, gr_col, 0.9])
    
    phis.append(0.4)
    phis.append(-0.4)
    ys.append(0.4)
    ys.append(-0.4)
    pts.append(0)
    pts.append(0)
    zs = [pt/j.perp() for pt in pts]
    zs_sized = [z*1000. for z in zs]
    cs.append([0,0,0,0])
    cs.append([0,0,0,0])

    plt.figure()
    # plt.scatter(phis, ys, c=colors, s=zs_sized, alpha=0.4, cmap="PuOr") #cmap='viridis')
    # plt.scatter(phis, ys, c=cs, s=zs_sized, alpha=0.4, cmap="magma") #cmap='viridis')
    plt.scatter(phis, ys, c=cs, s=zs_sized, alpha=0.4, cmap="magma") #cmap='viridis')
    
    plt.xlabel(r"$\Delta\varphi$")
    plt.ylabel(r"$\Delta y$")
    # plt.colorbar();  # show color scale
    plt.show()
    print(j.perp(), sjs[0].perp(), sjs[0].perp()/j.perp())
    # plt.rcdefaults()

In [ ]:
for n in range(5):
    j, sjs = next_event()
    # draw_jet(j)
    draw_subjets(j, sjs)
    # print ('[event] ', j.perp(), j, len(j.constituents()), len(sjs), [sj for sj in sjs])
    # print (' ')

In [ ]:
pythia_hard.stat()

In [ ]:
pythia_mb.stat()

In [ ]:
%jsroot on
cm = ROOT.TCanvas("cm","cm",600,400)
hmult_pup.SetLineColor(3)
hmult_pup.GetXaxis().SetTitle("charged particle multiplicity")
hmult_pup.GetYaxis().SetTitle("counts")
hmult_pup.Draw()
hmult_hard.SetLineColor(2)
hmult_hard.Draw("same")
cm.SetLogy()
cm.BuildLegend()
cm.Draw()

In [ ]:
%jsroot on
c = ROOT.TCanvas("c","c",1000,800)
c.Divide(2,2)
c.cd(1)
tj_delta.Draw("dpt:pt>>hdpt", "mpt>0.5", "colz")
hdpt = ROOT.gDirectory.Get("hdpt")
hdpt.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c.cd(2)
tj_delta.Draw("dL11:pt>>hdL11", "mpt>0.5", "colz")
hdL11 = ROOT.gDirectory.Get("hdL11")
hdL11.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c.cd(3)
tj_delta.Draw("dL21:pt>>hdL21", "mpt>0.5", "colz")
hdL21 = ROOT.gDirectory.Get("hdL21")
hdL21.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c.cd(4)
tj_delta.Draw("dL31:pt>>hdL31", "mpt>0.5", "colz")
hdL31 = ROOT.gDirectory.Get("hdL31")
hdL31.ProfileX().Draw("same")
ROOT.gPad.SetLogz()


c.Draw()

In [ ]:
%jsroot on
c1 = ROOT.TCanvas("c1","c1",1000,800)
c1.Divide(2,2)
c1.cd(1)
tj_delta.Draw("dpt:pt>>hdpt", "mpt>0.5 && pt>20 && pt<40", "colz")
hdpt = ROOT.gDirectory.Get("hdpt")
hdpt.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c1.cd(2)
tj_delta.Draw("dL11:L11>>pthdL11", "mpt>0.5 && pt>20 && pt<40", "colz")
pthdL11 = ROOT.gDirectory.Get("pthdL11")
pthdL11.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c1.cd(3)
tj_delta.Draw("dL21:L21>>pthdL21", "mpt>0.5 && pt>20 && pt<40", "colz")
pthdL21 = ROOT.gDirectory.Get("pthdL21")
pthdL21.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c1.cd(4)
tj_delta.Draw("dL31:L31>>pthdL31", "mpt>0.5 && pt>20 && pt<40", "colz")
pthdL31 = ROOT.gDirectory.Get("pthdL31")
pthdL31.ProfileX().Draw("same")
ROOT.gPad.SetLogz()


c1.Draw()

In [ ]:
%jsroot on
c2 = ROOT.TCanvas("c2","c2",1000,800)
c2.Divide(2,2)
c2.cd(1)
tj_delta.Draw("dpt:pt>>hdpt", "mpt>0.5 && pt>60 && pt<80", "colz")
hdpt = ROOT.gDirectory.Get("hdpt")
hdpt.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c2.cd(2)
tj_delta.Draw("dL11:L11>>pt2hdL11", "mpt>0.5 && pt>60 && pt<80", "colz")
pt2hdL11 = ROOT.gDirectory.Get("pt2hdL11")
pt2hdL11.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c2.cd(3)
tj_delta.Draw("dL21:L21>>pt2hdL21", "mpt>0.5 && pt>60 && pt<80", "colz")
pt2hdL21 = ROOT.gDirectory.Get("pt2hdL21")
pt2hdL21.ProfileX().Draw("same")
ROOT.gPad.SetLogz()

c2.cd(4)
tj_delta.Draw("dL31:L31>>pt2hdL31", "mpt>0.5 && pt>60 && pt<80", "colz")
pt2hdL31 = ROOT.gDirectory.Get("pt2hdL31")
pt2hdL31.ProfileX().Draw("same")
ROOT.gPad.SetLogz()


c2.Draw()

In [ ]:
%jsroot on
c3 = ROOT.TCanvas("c3","c3",1000,800)
c3.Divide(1,2)
c3.cd(1)
hpt_acc_pup.Draw("colz")
c3.cd(2)
hpt_acc_hard.Draw("colz")
c3.Draw()

In [ ]:
%jsroot on
tc = ROOT.TCanvas()
tj_delta.Draw("dpt:pt", "mpt>0.5", "colz")
ROOT.gPad.SetLogz()
tc.Draw()

In [ ]:
%jsroot on
tcL = ROOT.TCanvas()
tcL.Divide(2,2)
tcL.cd(1)
tj_delta.Draw("dL11:L11>>pt2hdL11x", "mpt>0.5 && pt>60 && pt<80", "colz")
pt2hdL11x = ROOT.gDirectory.Get("pt2hdL11x")

tj_delta.Draw("dL11:L11>>pt2hdL11xx", "mpt>0.5 && pt>60 && pt<80", "colz")
pt2hdL11xx = ROOT.gDirectory.Get("pt2hdL11xx")

pxx = pt2hdL11xx.ProjectionX()
pxx.SetLineStyle(2)
pxx.SetFillColorAlpha(2, 0.5)
pxx.SetFillStyle(1001)
pxx.Draw()


tcL.cd(2)
pyy = pt2hdL11xx.ProjectionY()
pyy.SetLineStyle(2)
pyy.SetFillColorAlpha(3, 0.5)
pyy.SetFillStyle(1001)
pyy.Draw()




#pt2hdL11x.ProjectionX().Draw()
#pxx.Draw("same")



ROOT.gPad.SetLogy()
ROOT.gPad.BuildLegend()
tcL.Draw()
